In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
conf = SparkConf().setMaster("local[1]").setAppName("ex39")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("ex39").getOrCreate()

25/01/10 11:51:40 WARN Utils: Your hostname, paloma-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/10 11:51:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/10 11:51:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/10 11:51:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/01/10 11:51:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
schema = "sensorId STRING, date STRING, PM10 FLOAT"
data = spark.read.csv("/home/paloma/Downloads/jupyter_spark/ex39/input.csv", header=False, schema=schema)
data.show()

+--------+----------+----+
|sensorId|      date|PM10|
+--------+----------+----+
|      s1|2016-01-01|20.5|
|      s2|2016-01-01|30.1|
|      s1|2016-01-02|60.2|
|      s2|2016-01-02|20.4|
|      s1|2016-01-03|55.5|
|      s2|2016-01-03|52.5|
|      s3|2016-01-03|12.5|
+--------+----------+----+



In [3]:
data = data.rdd # transforma em rdd
data.collect()

[Row(sensorId='s1', date='2016-01-01', PM10=20.5),
 Row(sensorId='s2', date='2016-01-01', PM10=30.100000381469727),
 Row(sensorId='s1', date='2016-01-02', PM10=60.20000076293945),
 Row(sensorId='s2', date='2016-01-02', PM10=20.399999618530273),
 Row(sensorId='s1', date='2016-01-03', PM10=55.5),
 Row(sensorId='s2', date='2016-01-03', PM10=52.5),
 Row(sensorId='s3', date='2016-01-03', PM10=12.5)]

In [4]:
gt_fifty = data.filter(lambda line: line.PM10 > 50)
gt_fifty = gt_fifty.map(lambda line: (line.sensorId, line.date))
gt_fifty.collect()

[('s1', '2016-01-02'), ('s1', '2016-01-03'), ('s2', '2016-01-03')]

In [7]:
dates = gt_fifty.groupByKey()
# groupByKey vai agrupar as datas pela key (no caso o id) e vai resultar numa lista com tuplas compostas
# por (id, colecao iteravel), tipo assim ('s1', <pyspark.resultiterable.ResultIterable at 0x706734de9bb0>)
dates.collect()

[('s1', <pyspark.resultiterable.ResultIterable at 0x706734de9bb0>),
 ('s2', <pyspark.resultiterable.ResultIterable at 0x706734de9b50>)]

In [8]:
dates = dates.mapValues(lambda lines: list(set(lines)))
# o mapValues para cada chave de cada tupla vai converter o iteravel para um set,
# removendo duplicatas e depois para lista
dates.collect()

[('s1', ['2016-01-02', '2016-01-03']), ('s2', ['2016-01-03'])]